# Imports

In [5]:
import requests
import json
import pandas as pd

# Definitions

In [6]:
leagueID = "1041760"
fpl_league_url = (
    f"https://fantasy.premierleague.com/api/leagues-classic/{leagueID}/standings/"
)

# Testing

## Connect

In [7]:
respose = requests.get(fpl_league_url)
if respose.status_code == 200:
    print("The API request was successful.")
    respose_json = respose.json()
else:
    print(f"Error: The API request failed with status code {respose.status_code}.")

The API request was successful.


## Parse respose_json

In [8]:
respose_json

{'new_entries': {'has_next': False, 'page': 1, 'results': []},
 'last_updated_data': '2023-12-16T22:22:57Z',
 'league': {'id': 1041760,
  'name': 'MNF',
  'created': '2023-08-07T09:06:50.015343Z',
  'closed': False,
  'max_entries': None,
  'league_type': 'x',
  'scoring': 'c',
  'admin_entry': 4651100,
  'start_event': 1,
  'code_privacy': 'p',
  'has_cup': True,
  'cup_league': None,
  'rank': None},
 'standings': {'has_next': False,
  'page': 1,
  'results': [{'id': 57395169,
    'event_total': 28,
    'player_name': 'Mason Gardener',
    'rank': 1,
    'last_rank': 1,
    'rank_sort': 1,
    'total': 962,
    'entry': 6963106,
    'entry_name': 'nice to Michu'},
   {'id': 35832424,
    'event_total': 28,
    'player_name': 'James Parfitt',
    'rank': 2,
    'last_rank': 3,
    'rank_sort': 2,
    'total': 953,
    'entry': 4651100,
    'entry_name': 'Parf 2.8'},
   {'id': 35939248,
    'event_total': 25,
    'player_name': 'Alex Winn',
    'rank': 3,
    'last_rank': 2,
    'rank_

In [9]:
league_df = pd.DataFrame(respose_json["standings"]["results"])
league_df

,id,event_total,player_name,rank,last_rank,rank_sort,total,entry,entry_name
0,57395169,28,Mason Gardener,1,1,1,962,6963106,nice to Michu
1,35832424,28,James Parfitt,2,3,2,953,4651100,Parf 2.8
2,35939248,25,Alex Winn,3,2,3,952,3794145,Al-Winn
3,58581266,35,Seb Winn,4,4,4,942,7052472,Gangsters Allardyce
4,56846765,16,Richard Collins,5,5,5,908,6911033,Eze-er said than Son
5,35879230,28,Joseph Parsons,6,6,6,884,4656770,AFC Great Name UTD
6,35832433,32,Ben Melhuish,7,8,7,884,4075412,Melshys Mediocres
7,35832437,41,Dan Pratlett,8,9,8,874,3194123,Onana what’s my name
8,35832439,10,Jacob Pickles,9,7,9,870,3148659,Gobblebox
9,48472859,19,Ben Wilson,10,10,10,854,6083206,Kloppenheimer
